In [11]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from torch.utils.data import DataLoader

import imageio.v3 as iio
from matplotlib.colors import Normalize

from lpfm.data.phys_dataset import SuperDataset
# Load all well datasets
from the_well.data import WellDataset
from lpfm.data.dataset_utils import get_datasets, get_dt_datasets

In [20]:
base_path = Path("/scratch/zsa8rk/org_datasets/datasets")
path = base_path / "rayleigh_benard" / "data" / "train"

dataset = WellDataset(str(path), full_trajectory_mode=True, use_normalization=True)

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

arr = dataset[0]["output_fields"]
# Convert PyTorch tensor to NumPy array
arr_np = arr.numpy()

n_channels = arr_np.shape[-1]

# Calculate global min and max for each channel
global_mins = np.min(arr_np, axis=(0, 1, 2))  # Min across time, height, width
global_maxs = np.max(arr_np, axis=(0, 1, 2))  # Max across time, height, width
print("Global min values:", global_mins)
print("Global max values:", global_maxs)

# Create figure with 2x2 subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 12))
axes = axes.flatten()  # Flatten for easier indexing

# Initialize images and colorbars for each channel
ims = []
for i in range(n_channels):
    im = axes[i].imshow(arr_np[0,:,:,i], cmap='viridis', vmin=global_mins[i], vmax=global_maxs[i])
    plt.colorbar(im, ax=axes[i], label=f'Channel {i}')
    axes[i].set_title(f'Channel {i} Data')
    ims.append(im)

# Adjust layout to prevent overlap
plt.tight_layout()

def update(frame):
    for i in range(n_channels):
        ims[i].set_array(arr_np[frame,:,:,i])
        axes[i].set_title(f'Channel {i} Data - Frame {frame}')
    return ims

# Create animation
ani = animation.FuncAnimation(fig, update, frames=arr_np.shape[0], 
                            interval=100, blit=True)

# Display the animation
plt.close()  # Close the initial figure
HTML(ani.to_jshtml())  # This will display the animation in the notebook

In [32]:
base_path = Path("/scratch/zsa8rk/datasets")
path = base_path / "rayleigh_benard_obstacle" / "data" / "train"

dataset = WellDataset(str(path), full_trajectory_mode=True, use_normalization=True, max_rollout_steps=100, min_dt_stride=10)

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

arr = dataset[0]["output_fields"]
# Convert PyTorch tensor to NumPy array
arr_np = arr.numpy()

n_channels = arr_np.shape[-1]

# Calculate global min and max for each channel
global_mins = np.nanmin(arr_np, axis=(0, 1, 2))  # Min across time, height, width
global_maxs = np.nanmax(arr_np, axis=(0, 1, 2))  # Max across time, height, width
print("Global min values:", global_mins)
print("Global max values:", global_maxs)

# Create figure with 2x2 subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 12))
axes = axes.flatten()  # Flatten for easier indexing

# Initialize images and colorbars for each channel
ims = []
for i in range(n_channels):
    im = axes[i].imshow(arr_np[0,:,:,i], cmap='viridis', vmin=global_mins[i], vmax=global_maxs[i])
    plt.colorbar(im, ax=axes[i], label=f'Channel {i}')
    axes[i].set_title(f'Channel {i} Data')
    ims.append(im)

# Adjust layout to prevent overlap
plt.tight_layout()

def update(frame):
    for i in range(n_channels):
        ims[i].set_array(arr_np[frame,:,:,i])
        axes[i].set_title(f'Channel {i} Data - Frame {frame}')
    return ims

# Create animation
ani = animation.FuncAnimation(fig, update, frames=arr_np.shape[0], 
                            interval=100, blit=True)

# Display the animation
plt.close()  # Close the initial figure
HTML(ani.to_jshtml())  # This will display the animation in the notebook

In [ ]:
base_path = Path("/hpcwork/rwth1802/coding/Large-Physics-Foundation-Model/data/datasets")
size = (256, 128)
field_names_t0 = ["t0_fields/temperature", "t0_fields/density", "t0_fields/pressure"]
field_names_t1 = ["t1_fields/velocity"]
def print_attrs(name, obj):
    print(f"\n{name}:")
    if isinstance(obj, h5py.Dataset):
        print(f"  Shape: {obj.shape}")
        print(f"  Dtype: {obj.dtype}")
    for key, val in obj.attrs.items():
        print(f"  {key}: {val}")

def check_size(name, obj):
    if isinstance(obj, h5py.Dataset):
        if obj.ndim == 4:
            if obj.shape[2] != size[0] or obj.shape[3] != size[1]:
                print(f"  Name: {name}")
                print(f"  Shape: {obj.shape}")
                print(f"  Dtype: {obj.dtype}")
            if name not in field_names_t0 and name not in field_names_t1:
                print(f"  Name: {name}")
                print(f"  Shape: {obj.shape}")
                print(f"  Dtype: {obj.dtype}")


for file in base_path.glob("**/*.hdf5"):
    # print(file.parent.name, file.name)
    with h5py.File(file, "r") as f:
    
        f.visititems(print_attrs)
        # f.visititems(check_size)

In [ ]:
def print_attrs(name, obj):
    print(f"\n{name}:")
    if isinstance(obj, h5py.Dataset):
        print(f"  Shape: {obj.shape}")
        print(f"  Dtype: {obj.dtype}")
    for key, val in obj.attrs.items():
        print(f"  {key}: {val}")

base_path = Path("/hpcwork/rwth1802/coding/Large-Physics-Foundation-Model/data/datasets/cylinder_pipe_flow_water/data/train")
fil_name = "cylinder_wall_flow_Re_20.0_cyl_dia_0.5.hdf5"
with h5py.File(base_path / fil_name, "r") as f:
    print_attrs("root", f)
    f.visititems(print_attrs)

In [6]:
base_path = Path("/scratch/zsa8rk/datasets")
datasets = [
    "cooled_object_pipe_flow_air",
    "heated_object_pipe_flow_air",
    "object_sym_flow_air",
    "object_sym_flow_water",
    "object_periodic_flow_water",
    "cylinder_sym_flow_water",
    "cylinder_pipe_flow_water",
    "euler_multi_quadrants_periodicBC",
    "rayleigh_benard",
    "shear_flow",
    "twophase_flow",
    "rayleigh_benard_obstacle",
    "acoustic_scattering_inclusions"
]

data_config = {
    "datasets": datasets,
    "data_dir": base_path,
    "n_steps_input": 4,
    "n_steps_output": 1,
    "dt_stride": [1,8],
    "out_shape": [256, 128],
    "max_samples_per_ds": None,
    "full_trajectory_mode": False,
    "zero_field_value": 0.0,
}

sets = get_dt_datasets(data_config, split="train")

In [ ]:
super_dataset = SuperDataset(sets)
print(len(super_dataset))
sets = get_dt_datasets(data_config, split="train")
super_dataset = SuperDataset(sets)
print(len(super_dataset))




In [9]:
data_config = {
    "datasets": ["twophase_flow"],
    "data_dir": "/hpcwork/rwth1802/coding/Large-Physics-Foundation-Model/data/datasets",
    "n_steps_input": 1,
    "n_steps_output": 1,
    "dt_stride": 1,
    "out_shape": [256, 128],
    "max_samples_per_ds": None,
    "max_rollout_steps": 500,
    "full_trajectory_mode": True,
}

loaded_datasets = get_datasets(data_config, split="train")
dataset = SuperDataset(loaded_datasets, (256, 128))

In [ ]:
time_idx = [0,1,2,3]
# Create figure with subplots
fig, axes = plt.subplots(5, len(time_idx), figsize=(15, 20))

# Plot all five physical fields for each timestep
field_names = ['Pressure', 'Density', 'Temperature', 'Velocity X', 'Velocity Y']
field_data = [pressure, density, temperature, vel_x, vel_y]
cmaps = ['plasma', 'viridis', 'inferno', 'cividis', 'magma']

for row, (field, name, cmap) in enumerate(zip(field_data, field_names, cmaps)):
    for col, t in enumerate(time_idx):
        im = axes[row, col].imshow(field[t,...], cmap=cmap)
        plt.colorbar(im, ax=axes[row, col], label=name)
        
        # Add labels and title
        axes[row, col].set_xlabel('X')
        axes[row, col].set_ylabel('Y')
        axes[row, col].set_title(f'{name} Field (t={time_idx[col]})')

plt.tight_layout()
plt.show()


In [ ]:
for traj in range(0,600,10):
    x, full_trajectory = dataset[traj]

    # # take first batch
    # x = x[0,...]
    # y = y[0,...]

    pressure = full_trajectory[..., 0]
    density = full_trajectory[..., 1]
    temperature = full_trajectory[..., 2]
    vel_x = full_trajectory[..., 3]
    vel_y = full_trajectory[..., 4]

    # vel_mag = np.linalg.norm(full_trajectory[:, :, :, 3:], axis=-1)

    # # reduce dim with size 1 to 0
    # vel_mag = np.squeeze(vel_mag)

    # # transpose x and y
    density = np.transpose(density, (0,  2, 1))

    # Create normalized colormap for consistent color scaling
    norm = Normalize(vmin=density.min(), vmax=density.max())

    # Create frames
    frames = []
    for t in range(density.shape[0]):
        # Create figure and plot
        fig, ax = plt.subplots()
        im = ax.imshow(density[t,:,:], norm=norm)
        plt.colorbar(im)
        ax.set_title(f'Time step {t}')
        
        # Convert plot to image array
        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (4,))[:,:,:3]
        frames.append(frame)
        plt.close()

    # Save as GIF
    output_path = Path(f"/hpcwork/rwth1802/coding/Large-Physics-Foundation-Model/data/datasets/twophase_flow/density_{traj}.gif")
    print(f"Writing {len(frames)} frames to {output_path}")
    iio.imwrite(output_path, frames, fps=30)
    print(f"Animation saved to {output_path}")